In [43]:
import os
import numpy as np
from google import genai
import pandas as pd
from tqdm import tqdm
import time
import json

os.environ["GOOGLE_API_KEY"] = ""
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

In [39]:
data = pd.read_csv('../data/twinviews-13k.csv')
l_list = data.l.to_list()
r_list = data.r.to_list()

In [40]:
BATCH_SIZE = 100   # puedes ajustar (500–1000 es seguro)
MODEL_NAME = "text-embedding-004"
SLEEP_TIME = 1.0

In [41]:
def embed_texts_in_batches(texts, model=MODEL_NAME, batch_size=BATCH_SIZE, sleep_time=SLEEP_TIME):
    """
    Genera embeddings por lotes con el modelo de Google AI.
    Devuelve una lista de vectores (cada embedding es una lista de floats).
    """
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        try:
            response = client.models.embed_content(
                model=model,
                contents=batch
            )
            batch_embeddings = [e.values for e in response.embeddings]
            all_embeddings.extend(batch_embeddings)
        except Exception as e:
            print(f"❌ Error en batch {i}-{i+batch_size}: {e}")
            # Esperar unos segundos y reintentar si falla
            time.sleep(5)
        time.sleep(sleep_time)

    return np.array(all_embeddings, dtype=np.float32)

In [42]:
print("🔹 Generando embeddings para columna L...")
l_embeddings = embed_texts_in_batches(l_list)
np.save("../data/l_embeddings.npy", l_embeddings)
print(f"Embeddings L guardados: {l_embeddings.shape}")

print("🔹 Generando embeddings para columna R...")
r_embeddings = embed_texts_in_batches(r_list)
np.save("../data/r_embeddings.npy", r_embeddings)
print(f"Embeddings R guardados: {r_embeddings.shape}")

print("🎯 Proceso completado. Archivos guardados en '../data/'")

🔹 Generando embeddings para columna L...


100%|██████████| 139/139 [05:43<00:00,  2.47s/it]


✅ Embeddings L guardados: (13855, 768)
🔹 Generando embeddings para columna R...


100%|██████████| 139/139 [05:21<00:00,  2.31s/it]


Embeddings R guardados: (13855, 768)
🎯 Proceso completado. Archivos guardados en '../data/'


In [45]:
# Lo guardamos también en formato parquet

def save_parquet(texts, embeddings, filename):
    df = pd.DataFrame({
        "text": texts,
        "embedding": [json.dumps(e.tolist()) for e in embeddings]  # 🔧 convertimos ndarray → list
    })
    df.to_parquet(filename, compression="snappy")

save_parquet(l_list, l_embeddings, "../data/l_embeddings.parquet")
save_parquet(r_list, r_embeddings, "../data/r_embeddings.parquet")

# Para cargar
#df = pd.read_parquet("../data/l_embeddings.parquet")
# df["embedding"] = df["embedding"].apply(json.loads)
